# Word2Vec using Unary Skip Gram Model

In [1]:
import tensorflow as tf
import os
import sys
import zipfile
import collections
from collections import Counter
import numpy as np
from six.moves import urllib
import random

### 下载数据

In [2]:
DOWNLOAD_URL   = "http://mattmahoney.net/dc/text8.zip"      # 下载文件的URL
DATA_FOLDER    = "./data/"                                  # 存放数据文件的文件夹路径
FILE_NAME      = "text8.zip"                                # 数据文件的名称
EXPECTED_BYTES = 31344016                                   # 文件的 bytes 大小

In [3]:
# 定义一个创建本地文件夹的函数
# 
# 参数
# path              : 创建路径
#
def make_dir(path):
    try:
        os.mkdir(path)
    except OSError:
        pass
    
# 获得当前文件的 bytes 大小
# 
# 参数
# file_path         : 文件的路径
#
# return            : 当前文件的大小
# 
def get_bytes(file_path):
    
    # 获得文件的描述性数据
    file_stats = os.stat(file_path)
    
    # 返回文件的大小
    return file_stats.st_size
    
# 检查数据的大小是否正确，用来检查是否下载了 “ 完整 ” 的数据集
# 如果文件大小不符合所期待的大小，则抛出异常
# 
# 参数
# file_path         : 文件路径
# expected_bytes    : 所期待的文件的大小
#
def check_bytes(file_path, expected_bytes):
    
    # 如果不符合期待的文件大小，则抛出异常
    assert get_bytes(file_path) == expected_bytes

In [4]:
# 定义一个下载数据的函数，并检查下载的数据是否完整的被下载了
#
# 参数
# source_url        : 文件 URL 下载路径
# download_folder   : 下载到本地 文件夹 的名字
# file_name         : 文件名
# expected_bytes    : 文件大小
#
# return            : 文件的路径
#
def download(download_url   = DOWNLOAD_URL, 
             data_folder    = DATA_FOLDER, 
             file_name      = FILE_NAME, 
             expected_bytes = EXPECTED_BYTES):
    
    # 如果下载数据的路径不存在的时候，则创建一个
    if not os.path.exists(data_folder):
        make_dir(data_folder)
        
    # 下载的数据的路径为文件夹的路径 + 文件名
    file_path = data_folder + file_name
    
    # 如果文件已经存在
    if os.path.exists(file_path):
        # 检查文件是否完整
        if get_bytes(file_path) == expected_bytes:
            # 如果完整，则返回该文件的路径，不做接下来的处理了
            print("Dataset already downloaded.")
            return file_path
        else:
            # 如果文件不完整，则删除文件，接下来重新下载一次
            os.remove(file_path)
    
    # 从网页上下载数据，下载文件可能会需要一段时间，请耐心等待
    print("Start downloading the data, the process may take several minutes, please be patient...")
    file_name, _ = urllib.request.urlretrieve( url = download_url, filename = file_path )
    
    # 检查下载的数据是否完整
    check_bytes( file_path, expected_bytes )
    
    # 返回文件的路径
    return file_path

In [5]:
# 下载数据
file_path = download()

# 检查数据的完整性
check_bytes( file_path, EXPECTED_BYTES )

# 下载成功
print("File downloaded at path", file_path)

Dataset already downloaded.
File downloaded at path ./data/text8.zip


### 读取数据

In [6]:
# 从zip中读取所有的单词
# 参数
# file_path         : ZIP 文件的路径
#
# return            : 该文件所包含的所有单词
#
def read_data(file_path):
    with zipfile.ZipFile( file = file_path ) as f:
        # namelist : 返回在压缩目录下的所有文件
        # read     : 读出文件的 bytes
        words = tf.compat.as_str( f.read( f.namelist()[0] ) ).split()
    return words

In [7]:
# 读取单词
words = read_data(file_path)

# 打印读取的单词的长度
print( "The whole content contains {} words.".format( len(words) ) )

# 打印最开始的5个单词
print( "The first 5 words are : {}.".format( words[:5]) )

The whole content contains 17005207 words.
The first 5 words are : ['anarchism', 'originated', 'as', 'a', 'term'].


### 构建数据集：将所有单词转换为index

In [8]:
VOCAB_SIZE = 50000    # 定义词库的大小为 50000

In [9]:
# 构建一个 word -> index 的 dictionary，
# 以及一个 index -> word 的 reverse_dictionary
# 参数
# words             : 单词输入，用来创建dictionary
# vocab_size        : 词库的大小
# 
# return 
#   word_index          :  输入的单词序列 转换成的 index 的序列
#                            [ 36, 1, 0, 998, ... , 12 ]
#   count               :  一个长度为 词库大小 的数组，每个数组的元素为 [word, count]
#                            [ ['UNK', 3456789], ['the', 12345], ['a',12344], ... , ]
#   dictionary          :  word -> index 的 dictionary
#   reverse_dictionary  :  index -> word 的 reverse_dictionary
#   
def build_dataset(words, vocab_size):
    
    dictionary = {}              # 初始化空词库
    count = [['UNK',-1]]         # 初始化 Unknown 的单词计数为-1
    
    # 找出出现最频繁的一组单词，加入到count数组中
    # 单词的个数为 词库的大小 减一，因为有一个位置已经被 unknown 占据了
    count.extend(Counter(words).most_common(vocab_size-1))
    
    index = 0                    # 用来记录每个单词在词库中的 index
    
    # 创建一个目录来存放前 1000 个单词的
    make_dir("processed")
    with open("processed/vocab_1000.tsv", "w") as f:
        for word, _ in count:
            # 遍历 count 来生成 word -> index 的 dictionary
            dictionary[word] = index
            # 将前 1000 个单词写入文件中
            if index < 1000:
                f.write(word + '\n')
            index += 1
    
    # 将所有的从 zip 文件中读取的单词，转换成相对应的 index
    # 如果单词存在于词库中，则返回它的index值
    # 否则，返回 0 -- UNK
    # 最终这个 len(word_index) == len(words)
    word_index = [dictionary[word] if word in dictionary else 0 for word in words]
    
    # word_index 中 0 出现的次数，即是 UNK 单词出现的次数
    count[0][1] = word_index.count(0)
    
    # zip  : zip 函数会对传入的 iterables 进行遍历和组合，
    #         最终返回 合并到一起 的 tuple 的数组
    #         # zip('ABCD', 'xy') --> Ax By
    #
    # dict : dict 函数可以从 tuple 的数组中生成一个新的dictionary
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    
    # 返回 单词的对应的 index 序列，每个单词对应出现的次数表，单词 - index 表， index - 单词 表
    return word_index, count, dictionary, reverse_dictionary

In [10]:
# 计算 word_index, count, dictionary, reverse_dictionary
word_index, count, dictionary, reverse_dictionary = build_dataset(words, VOCAB_SIZE)

# 可以节省内存
del words

# 查看一下嘴常见的 5 个单词
print("5 most common words are : {}.\n".format( count[:5] ) )

# 看一下 word_index 中的前十个 index 的值 和 对应的单词
print("10 first index and word are : {}.".format( ', '.join(
    str(index) + '/' +(reverse_dictionary[index]) for index in word_index[:10] ) ) )

5 most common words are : [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)].

10 first index and word are : 5234/anarchism, 3081/originated, 12/as, 6/a, 195/term, 2/of, 3134/abuse, 46/first, 59/used, 156/against.


### 生成训练的 batch

In [11]:
# 在单词的 index 序列中，随机生成一些用来训练的 batch
# 参数
# word_index        : 单词序列的的 index 表示，从中进行取样来生成 batch
# batch_size        : 训练 batch 的大小
# num_skip          : Skip Gram 模型中，从整个窗口中选取多少个不同的词作为output word
# window_size       : 取单词来预测的 window 大小
#
# return            :
#   batch           : 一个 batch size 的 input 数据
#   labels          : batch 中每个元素预测的单词
#
data_index = 0

def generate_batch(word_index, 
                   batch_size   = 8, 
                   num_skip     = 2, 
                   skip_window  = 1):
    
    global data_index                             # data_index 为全局变量，每一次取样都会对它的值进行更新
    
    assert batch_size % num_skip == 0             # 要求从中取出的单词数为 batch 的整除数
    assert num_skip <= 2 * skip_window            # 取出来的单词数要小于 2倍 window 的大小
    
    # shape 接收 int 的 tuple，生成多维的数组
    batch  = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size,1), dtype=np.int32)
    
    span = 2 * skip_window + 1                   # 左右两侧的大小为skip window 并 加上中间词 1
    buffer = collections.deque(maxlen=span)      # 创建一个大小为 span 的 buffer
    
    # 如果此时 buffer 取样越界了，则重头开始取样
    if data_index + span >= len(word_index):
        data_index = ( data_index + span ) % len(word_index)
        
    # 从 word_index 中读取一个长度为 span 的词序列放入 buffer 中，用来生成 batch
    buffer.extend(word_index[data_index:data_index+span])
    
    for i in range(batch_size // num_skip):        # batch_size/num_skip 是为了取到batch size的个数，我们所需中心词的个数
        
        # 左右两边的词为 context words
        context_words = [w for w in range(span) if span != skip_window]
        
        # 重洗一下这个 index 序列的顺序，用来随机选取 num_skip 个单词进行训练
        random.shuffle(context_words)
        words_to_use = collections.deque(context_words)
        for j in range(num_skip):
            context_word = words_to_use.pop()
            batch[i*num_skip+j]  = buffer[skip_window]
            labels[i*num_skip+j,0] = buffer[context_word]
        
        if data_index == len(word_index):       # 如果data_index已经到这个文档的末尾了，我们就从头开始
            buffer[:] = word_index[:span]
            data_index = span
        else:                                   # 把整个window往后面移一个位置
            buffer.append(word_index[data_index])
            data_index += 1
        
    return batch, labels

In [12]:
# 测试生成一下 batch 和 labels
batch, labels = generate_batch(word_index)

# 打印出来 input -> output
for i in range(8):
    print(i, batch[i], reverse_dictionary[batch[i]],
        '->', labels[i], reverse_dictionary[labels[i,0]])

0 3081 originated -> [5234] anarchism
1 3081 originated -> [3081] originated
2 12 as -> [5234] anarchism
3 12 as -> [3081] originated
4 5234 anarchism -> [3081] originated
5 5234 anarchism -> [12] as
6 3081 originated -> [12] as
7 3081 originated -> [5234] anarchism


### 定义模型

In [13]:
EMBBED_SIZE     = 64                 # 词向量的维数
NUM_SAMPLED     = 64                 # 进行 negative sampling 的取样个数
LEARNING_RATE   = 1.0                # 梯度下降的学习速率
BATCH_SIZE      = 128                # 每次训练的 batch 大小
VALIDATION_SIZE = 16                 # validation 单词个数，用来看一下这些取样单词之间的 similarity

In [14]:
graph = tf.Graph()

with graph.as_default():
    
    """
        Placeholders
    """
    
    # 训练时的输入 placeholder ：输入为中心词的 index 的 batch
    target_words  = tf.placeholder(name='target_words',  shape=[BATCH_SIZE],   dtype=tf.int32)
    # 训练时的输出 placeholder ：输出为 context word 的 index 的 batch
    context_words = tf.placeholder(name='context_words', shape=[BATCH_SIZE,1], dtype=tf.int32)
    
    """
        Variables
    """
    
    # 训练时不断的更新这个 embbding lookup table，得到最终的每个单词的词向量
    embeddings = tf.Variable(                      # 输入的 Lookup Table
        tf.random_uniform(                         # 随机生成的数据
            [VOCAB_SIZE,EMBBED_SIZE],              # tensor 的大小为 VOCAB_SIZE * EMBBED_SIZE
            -1.0, 1.0))                            # 范围在 -1.0 到 1.0 之间
    # 通过在 lookup table 中匹配中心词，找到的 target words 的词向量
    embed = tf.nn.embedding_lookup( embeddings, target_words )
    # 在训练中会用来更新的权值矩阵 W
    nce_weights = tf.Variable( 
        tf.truncated_normal(
            shape=[VOCAB_SIZE, EMBBED_SIZE], 
            stddev=1.0/(EMBBED_SIZE ** 0.5)) )
    # 在训练中用来更新的偏差矩阵 b
    nce_bais = tf.Variable(tf.zeros([VOCAB_SIZE]))
    
    """
        Loss function
    """
    
    # 用 negative sampling 方法来简化计算
    loss = tf.reduce_mean( 
        tf.nn.nce_loss(
            inputs=embed,                         # 输入为由 target words 在当前 lookup table 中找到的词向量
            labels=context_words,                 # 期待由 target words 能预测出来的 context words
            weights=nce_weights,                  # 权值矩阵
            biases=nce_bais,                      # 偏差矩阵
            num_classes=VOCAB_SIZE,               # 最终分类会得到的类别总数为 所有单词的个数
            num_sampled=NUM_SAMPLED) )            # 在 negative sampling 中的取样个数
    
    """
        Training step
    """
    
    # 利用梯度下降的方法来进行训练减小 loss
    train_step = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)
    
    """
        Normalized Embeddings
    """
    
    # 'x' is [[1, 1, 1]
    #         [1, 1, 1]]
    # tf.reduce_sum(x, 1, keep_dims=True) ==> [[3], [3]]
    # 这里的reduce sum，对每一个单词的 embbeding 词向量进行了求和，得到的一个有 VOCAB SIZE 个元素的数组
    # 其中每个元素的值为 开方 后的 所有元素的平方和 的一元矩阵
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    
    # 进行归一化处理的词向量 look up table，处理后，每一行的元素的平方和都为 1
    normalized_embeddings = embeddings / norm

### 训练模型

In [15]:
BATCH_SIZE      = 128           # 训练 batch 的大小
NUM_SKIP        = 2             # Skip Gram 模型中，从整个窗口中选取多少个不同的词作为output word
SKIP_WINDOW     = 1             # 取单词来预测的 window 大小
NUM_TRAIN_STEPS = 40000         # 总共训练的次数
WEIGHTS_FOLDER  = 'processed'   # 权值最后存储的位置

In [16]:
with tf.Session(graph=graph) as sess:
    
    # 初始化在 graph 中的所有变量
    sess.run(tf.global_variables_initializer())
    
    # 总的损失
    total_loss = 0.0
    
    # 一步一步的训练模型
    for step in range(NUM_TRAIN_STEPS):
        
        # 生成 input - target batch
        target_words_batch, context_words_batch = generate_batch(batch_size=BATCH_SIZE, word_index=word_index)
        
        # 运行一次 Gradient Descent 来降低 loss
        _, batch_loss = sess.run([train_step, loss], feed_dict={
            target_words: target_words_batch,
            context_words: context_words_batch
        })
        total_loss += batch_loss
        
        # 每 2000 次运算，打印一下在之前 2000 次计算中的 loss 的平均值
        if (step+1) % 2000 == 0:
            print("Average loss at step {}: {:5.1f}".format(step+1, total_loss / 2000))
            total_loss = 0.0
    
    final_embeddings = normalized_embeddings.eval()

Average loss at step 2000: 113.6
Average loss at step 4000:  52.0
Average loss at step 6000:  32.6
Average loss at step 8000:  22.3
Average loss at step 10000:  17.2
Average loss at step 12000:  13.8
Average loss at step 14000:  11.2
Average loss at step 16000:   9.4
Average loss at step 18000:   7.8
Average loss at step 20000:   7.3
Average loss at step 22000:   6.3
Average loss at step 24000:   6.2
Average loss at step 26000:   6.0
Average loss at step 28000:   5.7
Average loss at step 30000:   5.2
Average loss at step 32000:   5.2
Average loss at step 34000:   5.0
Average loss at step 36000:   5.0
Average loss at step 38000:   4.7
Average loss at step 40000:   4.6


In [17]:
final_embeddings.shape

(50000, 64)

### 找到最相似的单词

In [18]:
# 找到最接近的单词
def find_nearest_words(word,                                          
                       num_nearest_words=20,                          # 想要找到多少个最相近相似的词
                       embeddings=final_embeddings,                   # embeddings 的 lookup table
                       dictionary=dictionary,                         # word - index 字典
                       reverse_dictionary=reverse_dictionary,         # index - word 字典反转
                       only_comapre_with_frequent_word=True):         # 只在最常见的词中找相似词么
    
    # to lower case 输入 word
    if word != word.lower():
        print('{} will be treated as lower case {}'.format(word, word.lower()))
        word = word.lower()
    
    # 处理异常单词
    if word not in dictionary:
        print('{} is not recognized in dictionary, treat as UNK'.format(word))
        word = 'UNK'
    
    # Find Word Embedding of Target Word 
    embed = [ tf.nn.embedding_lookup( embeddings, dictionary[word] ) ]
    
    # Compute the similarity of this word with all other words in dictionary
    similarity = tf.matmul(embeddings, embed, transpose_b=True)
    
    with tf.Session() as sess:
        sin = similarity.eval()
        # 如果只考虑最频繁的1000个单词
        if only_comapre_with_frequent_word:
            sin = sin[:1000]
        # 按降序进行排序相似度，并将排序后的index放到 sorted index 变量中
        sorted_index = sorted(range(len(sin)), key=lambda k: sin[k], reverse=True)
        # 将 单词 和 相似度 放在一个 array 中
        nearest_words_similarity = [
            [reverse_dictionary[index], sin[index][0]] for index in sorted_index[1:num_nearest_words]]
    
    # Print out result
    print("For word '{}', the {} nearest words are : \n".format(word, num_nearest_words))
    for word, similarity in nearest_words_similarity:
        print("{} \t: {}".format(word, similarity))

In [19]:
find_nearest_words("computer")

For word 'computer', the 20 nearest words are : 

designed 	: 0.5785603523254395
blue 	: 0.5701479315757751
function 	: 0.569938063621521
great 	: 0.5639290809631348
types 	: 0.5579832792282104
zero 	: 0.530914843082428
chinese 	: 0.5194635987281799
revolution 	: 0.5166011452674866
letters 	: 0.5131276249885559
date 	: 0.5068237781524658
collection 	: 0.5063639283180237
release 	: 0.4990079998970032
if 	: 0.4969874620437622
players 	: 0.4943467974662781
due 	: 0.4907388687133789
southern 	: 0.48938414454460144
left 	: 0.4888436794281006
where 	: 0.48667147755622864
he 	: 0.4859670400619507


### 找到与单词对类似的另外一个单词

In [43]:
def find_similar_pair_word(word_pair,                                     # 用来寻找与此 word pair 相似的 word pair
                           word,                                          # 给一个词，找与他对应的词
                           embeddings=final_embeddings,                   # 词向量的 lookup table
                           dictionary=dictionary,                         # word - index 字典
                           reverse_dictionary=reverse_dictionary):        # index - word 字典反转)
    
    # 检查 word pair 的输入是否合理
    assert type(word_pair) is list
    assert len (word_pair) == 2
    
    # 寻找 word pair 中两个单词所对应的词向量
    word_1 = word_pair[0]
    word_2 = word_pair[1]
    
    if word_1 not in dictionary:
        word_1 = "UNK"
    if word_2 not in dictionary:
        word_2 = "UNK"
        
    embedding_1 = tf.nn.embedding_lookup( embeddings, dictionary[word_1] )
    embedding_2 = tf.nn.embedding_lookup( embeddings, dictionary[word_2] )
    
    # 计算 word pair 两个单词之间的 cosine 距离
    word_pair_cosine_distance = tf.matmul( [embedding_1], [embedding_2], transpose_b=True )
    
    # 对于用来寻找 pair word 的单词，找到它的词向量，已经它与所有单词的 cosine 相似度
    if word not in dictionary:
        word = "UNK"
    
    embed = [tf.nn.embedding_lookup( embeddings, dictionary[word] )]
    similarity = tf.matmul(embeddings, embed, transpose_b=True)

    with tf.Session() as sess:
        
        # 得到输入 word pair 的 cosine 距离
        distance = word_pair_cosine_distance.eval()[0][0]
        print("Word pair's cosine distance is :",distance)
        
        # 求单词与词库所有单词的 similarity 与目标 distance 之间的差异的绝对值，绝对值越小，则说明该单词成对的可能性越大
        pair_distance = tf.abs(similarity - distance).eval()
        # 按升序排列
        sorted_index = sorted(range(len(pair_distance)), key=lambda k: pair_distance[k])
        # 取前 20 个 单词作为可能的 word pair 单词
        prob_words = [reverse_dictionary[index] for index in sorted_index[:20]]
        print("Possible {}'s pair words are :".format(word))
        for word in prob_words:
            print("Word: {} \t Different distance with word pair: {}".format(word, pair_distance[dictionary[word]][0]))

In [45]:
find_similar_pair_word(['am','was'], 'go')

Word pair's cosine distance is : 0.387206
Possible go's pair words are :
Word: subdued 	 Different distance with word pair: 3.7550926208496094e-06
Word: cristina 	 Different distance with word pair: 8.07642936706543e-06
Word: calibre 	 Different distance with word pair: 2.5093555450439453e-05
Word: parachute 	 Different distance with word pair: 2.6285648345947266e-05
Word: picked 	 Different distance with word pair: 2.7418136596679688e-05
Word: tartan 	 Different distance with word pair: 3.3527612686157227e-05
Word: abhidharma 	 Different distance with word pair: 3.55839729309082e-05
Word: physician 	 Different distance with word pair: 3.93986701965332e-05
Word: shift 	 Different distance with word pair: 4.023313522338867e-05
Word: magistrates 	 Different distance with word pair: 4.51207160949707e-05
Word: roadster 	 Different distance with word pair: 4.8041343688964844e-05
Word: fullness 	 Different distance with word pair: 5.2928924560546875e-05
Word: apl 	 Different distance with wo